In [ ]:
# =========================
# Notebook 3 — Final dataset construction (ROOT-ONLY, EMR-aligned)
# =========================
import warnings
from pathlib import Path
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=FutureWarning)

CONFIG = {
    "DATE_START": "2021-01-01",
    "DATE_END": "2023-12-31",
    "TICKERS": ["AAPL", "AMZN", "MSFT", "TSLA", "AMD"],

    # Prices (calendar source of truth)
    "PRICE_TIDY": "historical_stock_yfinance_full_2021_2023_tidy.csv",

    # NB2 daily sentiment inputs (authoritative)
    "TW_DAILY": "twitter_daily.csv",
    "RD_DAILY": "reddit_daily.csv",
    "NW_DAILY": "sp500_daily.csv",

    # Directories
    "INT_DIR": "intermediates",     # fallback only
    "OUT_DIR": "final_inputs",

    # Feature name prefixes
    "ALLOWED_PREFIXES": ("Tw_", "Rd_", "Nw_SP500_"),
}

DATE_START_TS = pd.to_datetime(CONFIG["DATE_START"])
DATE_END_TS   = pd.to_datetime(CONFIG["DATE_END"])
TICKERS = CONFIG["TICKERS"]

OUT_DIR = Path(CONFIG["OUT_DIR"]); OUT_DIR.mkdir(parents=True, exist_ok=True)
print("Notebook 3 config OK.")

# =========================
# Pre-flight + tidy loaders
# =========================
def find_daily(name: str) -> Path | None:
    p_root = Path(name)
    p_int  = Path(CONFIG["INT_DIR"]) / name
    if p_root.exists(): return p_root
    if p_int.exists():  return p_int
    return None

def load_prices_tidy() -> pd.DataFrame:
    p = find_daily(CONFIG["PRICE_TIDY"]) or Path(CONFIG["PRICE_TIDY"])
    if not p.exists():
        raise FileNotFoundError(f"Missing tidy price file: {CONFIG['PRICE_TIDY']}")
    px = pd.read_csv(p, parse_dates=["Date"])
    if "Ticker" in px.columns:
        px = px.rename(columns={"Date":"date","Ticker":"ticker"})
    else:
        px = px.rename(columns={"Date":"date"})
        if "ticker" not in px.columns and "Symbol" in px.columns:
            px = px.rename(columns={"Symbol":"ticker"})
    px = px[px["ticker"].isin(TICKERS)]
    px = px[(px["date"] >= DATE_START_TS) & (px["date"] <= DATE_END_TS)]
    return px.sort_values(["ticker","date"]).reset_index(drop=True)

SRC_FILES = [("Twitter", CONFIG["TW_DAILY"]),
             ("Reddit",  CONFIG["RD_DAILY"]),
             ("SP500",   CONFIG["NW_DAILY"])]

def load_sentiment_long(tickers) -> pd.DataFrame:
    """Load NB2 daily files. If they lack 'source', synthesise it from filename and
    convert wide prefixed cols to tidy rows for a clean pivot later."""
    frames = []
    for src_name, fname in SRC_FILES:
        p = find_daily(fname)
        if p is None:
            print(f"Missing daily file: {fname} (skipping)")
            continue
        df = pd.read_csv(p, parse_dates=["date"])
        df = df[df["ticker"].isin(tickers)].copy()

        # Preferred: tidy already has a 'source' column
        if "source" in df.columns:
            df["source"] = df["source"].replace({"News":"SP500"})  # normalise naming
            frames.append(df)
            continue

        # Fallback: wide with a known prefix -> synthesise tidy rows
        pref = {"Twitter":"Tw_","Reddit":"Rd_","SP500":"Nw_SP500_"}[src_name]
        feat_cols = [c for c in df.columns if c.startswith(pref)]
        if not feat_cols:
            print(f"{fname}: no {pref}* columns found; skipping")
            continue
        tmp = df[["date","ticker"] + feat_cols].copy()
        tmp.columns = ["date","ticker"] + [c.replace(pref,"") for c in feat_cols]
        tmp.insert(2, "source", src_name)
        frames.append(tmp)

    if not frames:
        return pd.DataFrame(columns=["date","ticker","source"])

    long_df = pd.concat(frames, ignore_index=True)
    long_df = long_df.sort_values(["ticker","date","source"]).reset_index(drop=True)
    return long_df

def pivot_sentiment(long_df: pd.DataFrame) -> pd.DataFrame:
    """Pivot tidy sentiment to a single wide frame keyed by (date, ticker)."""
    if long_df.empty:
        return pd.DataFrame(columns=["date","ticker"])
    feats = [c for c in long_df.columns if c not in ["date","ticker","source"]]
    # NB2 already has one row per (date,ticker,source); mean is idempotent
    gv = long_df.groupby(["date","ticker","source"], as_index=False)[feats].mean()
    piv = gv.pivot(index=["date","ticker"], columns="source", values=feats)
    pref_map = {"Twitter":"Tw_","Reddit":"Rd_","SP500":"Nw_SP500_"}
    piv.columns = [f"{pref_map[src]}{feat}" for feat, src in piv.columns]
    piv = piv.reset_index().sort_values(["ticker","date"]).reset_index(drop=True)
    return piv

prices = load_prices_tidy()
all_sent_long = load_sentiment_long(TICKERS)
all_sent_wide = pivot_sentiment(all_sent_long)
print("Loaded prices:", prices.shape, "| tidy sentiment:", all_sent_long.shape, "| wide pivot:", all_sent_wide.shape)

# =========================
# Technical indicators (RSI-14, MACD 12-26-9, returns, vols, SMAs)
# =========================
def _rsi_wilder(close: pd.Series, period: int = 14) -> pd.Series:
    delta = close.diff()
    up, down = delta.clip(lower=0), -delta.clip(upper=0)
    roll_up = up.ewm(alpha=1/period, adjust=False).mean()
    roll_down = down.ewm(alpha=1/period, adjust=False).mean()
    rs = roll_up / roll_down.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(0.0)

def _macd(close: pd.Series, fast=12, slow=26, signal=9) -> tuple[pd.Series,pd.Series]:
    ema_fast = close.ewm(span=fast, adjust=False).mean()
    ema_slow = close.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    sig  = macd.ewm(span=signal, adjust=False).mean()
    return macd, sig

def compute_technicals(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values("date").reset_index(drop=True).copy()
    df["Return_1d"] = df["Close"].pct_change()
    df["LogRet_1d"] = np.log(df["Close"]).diff()
    df["Vol_7"]  = df["LogRet_1d"].rolling(7,  min_periods=7).std()
    df["Vol_21"] = df["LogRet_1d"].rolling(21, min_periods=21).std()
    df["SMA_7"]  = df["Close"].rolling(7,  min_periods=7).mean()
    df["SMA_21"] = df["Close"].rolling(21, min_periods=21).mean()
    df["RSI_14"] = _rsi_wilder(df["Close"], 14)
    macd, sig = _macd(df["Close"], 12, 26, 9)
    df["MACD"] = macd
    df["MACD_Signal"] = sig
    return df

# =========================
# Build final per-ticker (single merge from NB2 daily)
# =========================
def build_final_for_ticker(ticker: str,
                           prices_df: pd.DataFrame,
                           sent_wide_df: pd.DataFrame) -> pd.DataFrame:
    p = prices_df[prices_df["ticker"] == ticker].copy()
    p = p[(p["date"] >= DATE_START_TS) & (p["date"] <= DATE_END_TS)]
    p = compute_technicals(p)

    s = sent_wide_df[sent_wide_df["ticker"] == ticker].drop(columns=["ticker"], errors="ignore")
    df = p.merge(s, on="date", how="left")

    # Fill sentiment NaNs neutrally (NB2 already calendar-fills; this is defensive)
    sent_cols = [c for c in df.columns if c.startswith(CONFIG["ALLOWED_PREFIXES"])]
    if sent_cols:
        df[sent_cols] = df[sent_cols].fillna(0.0)

    # REQUIRED FIX: recompute *_zero_day strictly from *_count_items
    for pref in ["Tw_", "Rd_", "Nw_SP500_"]:
        cnt = f"{pref}count_items"
        zro = f"{pref}zero_day"
        if cnt in df.columns:
            df[zro] = (df[cnt] == 0).astype(float)

    # Target is next trading day's Close (leakage-safe)
    df["Target"] = df["Close"].shift(-1)

    # Drop rows where any numeric feature or Target is NaN (warm-up trimming)
    num_cols = df.select_dtypes(include=[np.number]).columns
    df = df.dropna(subset=["Target"]).dropna(subset=num_cols)

    # Column ordering
    core = ["date","ticker",
            "Open","High","Low","Close","Volume",
            "Return_1d","LogRet_1d","Vol_7","Vol_21","SMA_7","SMA_21","RSI_14","MACD","MACD_Signal"]
    others = [c for c in df.columns if c not in core + ["Target"]]
    ordered = core + sorted([c for c in others if c.startswith(CONFIG["ALLOWED_PREFIXES"])]) + ["Target"]
    df = df[[c for c in ordered if c in df.columns]]

    out_path = OUT_DIR / f"{ticker}_input.csv"
    df.to_csv(out_path, index=False)
    print(f"{ticker}: wrote {out_path} rows={len(df)} cols={len(df.columns)}")
    return df

# =========================
# Run for all tickers
# =========================
built = {}
for t in TICKERS:
    built[t] = build_final_for_ticker(t, prices, all_sent_wide)
print("\nAll final datasets constructed.")

# =========================
# Sanity: schema, leakage guard, NaNs, coverage, prefixes
# =========================
rows = []
need_core = {"date","ticker","Open","High","Low","Close","Volume",
             "Return_1d","LogRet_1d","Vol_7","Vol_21","SMA_7","SMA_21","RSI_14","MACD","MACD_Signal","Target"}

for t in TICKERS:
    p = OUT_DIR / f"{t}_input.csv"
    if not p.exists():
        rows.append([t,"missing",None,None,0,0,0,"",""])
        continue
    df = pd.read_csv(p, parse_dates=["date"])

    dup = int(df.duplicated(["date"]).sum())
    sorted_ok = bool(df["date"].is_monotonic_increasing)
    close_next = df["Close"].shift(-1)
    mis = int(((df["Target"] - close_next).abs() > 1e-9).sum())
    if mis not in (0,1): print(f"WARNING {t}: target mismatch rows = {mis}")

    total_nas = int(df.select_dtypes(include=[np.number]).isna().sum().sum())
    start = df["date"].min().date() if not df.empty else None
    end   = df["date"].max().date() if not df.empty else None

    unexpected = [c for c in df.columns
                  if not any(c.startswith(a) for a in CONFIG["ALLOWED_PREFIXES"])
                  and c not in need_core]
    missing_core = sorted(list(need_core - set(df.columns)))

    ok = (dup == 0) and sorted_ok and (total_nas == 0) and (len(missing_core) == 0)
    rows.append([t, "ok" if ok else "check", start, end, len(df), dup, total_nas,
                 ";".join(missing_core), ";".join(unexpected)])

rep = pd.DataFrame(rows, columns=["Ticker","Status","Start","End","Rows","DuplicateDates","TotalNaNs","MissingCore","Unexpected"])
print(rep.to_string(index=False))

SANITY = Path("sanity_reports"); SANITY.mkdir(parents=True, exist_ok=True)
rep.to_csv(SANITY / "notebook03_final_inputs_summary.csv", index=False)
print("\nSaved:", (SANITY / "notebook03_final_inputs_summary.csv").resolve())

# =========================
# Quick visibility: first non-zero Twitter rows per ticker (if present)
# =========================
for t in TICKERS:
    p = OUT_DIR / f"{t}_input.csv"
    if not p.exists():
        continue
    df = pd.read_csv(p, parse_dates=["date"])
    twc = "Tw_count_items"
    if twc in df.columns:
        nz = df[df[twc] > 0].head(5)
        print(f"\n{t} first non-zero Twitter rows in final_inputs:")
        print(nz[["date", twc, "Tw_mean_s"]].to_string(index=False) if not nz.empty else "  (none)")

# =========================
# Cross-source key alignment (debug only)
# =========================
def _keyset(df):
    return set(zip(df["ticker"], df["date"].dt.strftime("%Y-%m-%d"))) if not df.empty else set()

k_long = _keyset(all_sent_long)  # set collapse is fine for debug
k_wide = _keyset(all_sent_wide)
print("Sentiment tidy empty:", all_sent_long.empty, "| wide empty:", all_sent_wide.empty)
print("Note: tidy is per (ticker,date,source); pivot merges sources per (ticker,date).")

# =========================
# NB3 Splits: Build from inputs + Audit (self-contained)
# =========================
from pathlib import Path

# ---- Settings ----
TICKERS = ["AAPL","AMZN","MSFT","TSLA","AMD"]

# Locations
try:
    inputs_dir = Path(OUT_DIR)
except NameError:
    inputs_dir = Path("final_inputs") if Path("final_inputs").exists() else Path(".")

splits_out_dir = Path(".")  # keep as '.' unless ARIMA loader updated

DATE_COL, SET_COL = "date", "Set"

# Chronological split windows (authoritative)
TRAIN_START, TRAIN_END = "2021-02-02","2022-12-31"
VAL_START,   VAL_END   = "2023-01-01","2023-05-31"
TEST_START,  TEST_END  = "2023-06-01","2023-12-28"
EXPECTED_TEST_LEN = 146

def build_nb3_splits(df: pd.DataFrame) -> pd.DataFrame:
    d = df[[DATE_COL]].copy()
    d[SET_COL] = None
    d.loc[(d[DATE_COL] >= TRAIN_START) & (d[DATE_COL] <= TRAIN_END), SET_COL] = "Train"
    d.loc[(d[DATE_COL] >= VAL_START)   & (d[DATE_COL] <= VAL_END),   SET_COL] = "Val"
    d.loc[(d[DATE_COL] >= TEST_START)  & (d[DATE_COL] <= TEST_END),  SET_COL] = "Test"
    d = d.dropna(subset=[SET_COL]).sort_values(DATE_COL).reset_index(drop=True)
    return d

def audit_nb3(df_input: pd.DataFrame, df_split: pd.DataFrame, ticker: str) -> dict:
    if df_split.duplicated(DATE_COL).any():
        raise ValueError(f"{ticker}: duplicate dates in splits file")
    if not set(df_split[DATE_COL]).issubset(set(df_input[DATE_COL])):
        raise ValueError(f"{ticker}: splits contain dates not present in input")

    g = df_split.groupby(SET_COL)[DATE_COL].agg(['min','max','count'])
    tr = g.loc["Train"] if "Train" in g.index else None
    va = g.loc["Val"]   if "Val"   in g.index else None
    te = g.loc["Test"]  if "Test"  in g.index else None
    if tr is None or va is None or te is None:
        raise ValueError(f"{ticker}: Train/Val/Test not all present")

    te_min, te_max, te_n = str(te['min'].date()), str(te['max'].date()), int(te['count'])
    if te_min != TEST_START:  raise AssertionError(f"{ticker}: Test start {te_min} != {TEST_START}")
    if te_max != TEST_END:    raise AssertionError(f"{ticker}: Test end {te_max} != {TEST_END}")
    if te_n != EXPECTED_TEST_LEN: raise AssertionError(f"{ticker}: Test rows {te_n} != {EXPECTED_TEST_LEN}")

    tr_min, tr_max = str(tr['min'].date()), str(tr['max'].date())
    va_min, va_max = str(va['min'].date()), str(va['max'].date())
    if not (TRAIN_START <= tr_min <= TRAIN_END and TRAIN_START <= tr_max <= TRAIN_END):
        raise AssertionError(f"{ticker}: Train span {tr_min}..{tr_max} not within {TRAIN_START}..{TRAIN_END}")
    if not (VAL_START   <= va_min <= VAL_END   and VAL_START   <= va_max <= VAL_END):
        raise AssertionError(f"{ticker}: Val span {va_min}..{va_max} not within {VAL_START}..{VAL_END}")

    return {
        "ticker": ticker,
        "train_min": tr_min, "train_max": tr_max, "train_n": int(tr['count']),
        "val_min": va_min,   "val_max": va_max,   "val_n":   int(va['count']),
        "test_min": te_min,  "test_max": te_max,  "test_n":  te_n
    }

summary_rows = []
for t in TICKERS:
    fi = inputs_dir / f"{t}_input.csv"
    assert fi.exists(), f"Missing input: {fi}"
    df = pd.read_csv(fi, parse_dates=[DATE_COL]).sort_values(DATE_COL)
    assert {DATE_COL, "Close"}.issubset(df.columns), f"{fi} must contain 'date' and 'Close'"

    splits = build_nb3_splits(df)

    out_path = splits_out_dir / f"splits_{t}.csv"
    if out_path.exists():
        out_path.rename(out_path.with_suffix(out_path.suffix + ".bak"))

    splits.to_csv(out_path, index=False)
    audit_row = audit_nb3(df, splits, t)
    summary_rows.append(audit_row)
    print(f"{t}: wrote {out_path} | Train={audit_row['train_n']}, Val={audit_row['val_n']}, Test={audit_row['test_n']}")

summary = pd.DataFrame(summary_rows)
summary.to_csv("appendix_split_checks.csv", index=False)
print("\nNB3 split summary")
print(summary.to_string(index=False))
print("\nAll splits are compliant.")

# =========================
# Optional integrity hashes for inputs and splits (appendix_file_hashes.csv)
# =========================
import os, hashlib

def sha256_of(path: Path) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

rows, missing = [], []
for t in TICKERS:
    for p in [Path(OUT_DIR) / f"{t}_input.csv", Path(f"splits_{t}.csv")]:
        if p.exists():
            rows.append({"file": str(p), "bytes": p.stat().st_size, "sha256": sha256_of(p)})
        else:
            missing.append(str(p))
            rows.append({"file": str(p), "bytes": None, "sha256": None})

pd.DataFrame(rows).to_csv("appendix_file_hashes.csv", index=False)
print("Wrote appendix_file_hashes.csv")
if missing:
    print("Missing files (hashes not computed):")
    for m in missing:
        print(" -", m)

# =========================
# Zip NB3 inputs and splits for export
# =========================
import zipfile

TICKERS = ["AAPL","AMZN","MSFT","TSLA","AMD"]
inputs_dir = Path("final_inputs") if Path("final_inputs").exists() else Path(".")
splits_dir = Path(".")  # splits saved in CWD above

fin_zip = Path("final_inputs.zip")
with zipfile.ZipFile(fin_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for t in TICKERS:
        p = inputs_dir / f"{t}_input.csv"
        if p.exists():
            zf.write(p, arcname=Path("final_inputs") / p.name)
        else:
            print(f"[warn] missing input: {p}")

spl_zip = Path("splits.zip")
with zipfile.ZipFile(spl_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    candidates = sorted(Path(".").glob("splits_*.csv"))
    if not candidates:
        print("[warn] no splits_*.csv found")
    for p in candidates:
        zf.write(p, arcname=Path("splits") / p.name)

print("Created:", fin_zip.resolve())
print("Created:", spl_zip.resolve())

# Download in Colab (best-effort)
try:
    from google.colab import files as colab_files
    colab_files.download(str(fin_zip))
    colab_files.download(str(spl_zip))
except Exception:
    try:
        from IPython.display import FileLink, display
        display(FileLink(str(fin_zip)))
        display(FileLink(str(spl_zip)))
        print("Click the links above to download the zips.")
    except Exception:
        pass

Notebook 3 config OK.
Loaded prices: (3765, 8) | tidy sentiment: (11295, 23) | wide pivot: (3765, 62)
AAPL: wrote final_inputs/AAPL_input.csv rows=731 cols=77
AMZN: wrote final_inputs/AMZN_input.csv rows=731 cols=77
MSFT: wrote final_inputs/MSFT_input.csv rows=731 cols=77
TSLA: wrote final_inputs/TSLA_input.csv rows=731 cols=77
AMD: wrote final_inputs/AMD_input.csv rows=731 cols=77

All final datasets constructed.
Ticker Status      Start        End  Rows  DuplicateDates  TotalNaNs MissingCore Unexpected
  AAPL     ok 2021-02-03 2023-12-28   731               0          0                       
  AMZN     ok 2021-02-03 2023-12-28   731               0          0                       
  MSFT     ok 2021-02-03 2023-12-28   731               0          0                       
  TSLA     ok 2021-02-03 2023-12-28   731               0          0                       
   AMD     ok 2021-02-03 2023-12-28   731               0          0                       

Saved: /content/sanity_report

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>